<a href="https://colab.research.google.com/github/danielsaggau/Elections-Belarus/blob/main/Belarus_ML_Full_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rpy2
%load_ext rpy2.ipython

In [ ]:
import pandas as pd
import numpy as np
df_pls=pd.read_excel("df_merge_n.xlsx")


In [ ]:
df_pls_pls=df_pls.drop(columns=['Unnamed: 0'])
df_pls_pls=df_pls_pls.fillna(0)
df_pls_new=df_pls_pls.drop(columns=['commission_code','number_of_cite'])
df_pls_new.info()

In [ ]:
df_merge1=df_pls_new.drop(columns=['attachment1','attachment2','attachment3','attachment4','attachment5','comment','id','type','name','location','origin','area'])
df_merge1.info()

In [ ]:
df_merge1['lukashenko_share']=df_merge1['Lukashenko']/df_merge1['number_of_voters_who_took_part_in_the_voting']

In [ ]:
df_merge1['lukashenko_share']=df_merge1['lukashenko_share'].replace([np.inf, -np.inf], 0)
df_merge1.info()

In [ ]:
df_merge2=df_merge1.drop(columns=['parent_id','description_x','commission_id','work_title','description_y','city_or_district_within_the_region','area_in_the_city','settlement'])

In [ ]:
df_merge2.columns

In [ ]:
df_merge2['region'].unique()

In [ ]:
df_merge2['region']=df_merge2['region'].replace('Брестская','brestskaya')
df_merge2['region']=df_merge2['region'].replace('Витебская','vitebskaya')
df_merge2['region']=df_merge2['region'].replace('Гомельская','gomelskaya')
df_merge2['region']=df_merge2['region'].replace('Гродненская','grodnenskaya')
df_merge2['region']=df_merge2['region'].replace('Минская','miskaya')
df_merge2['region']=df_merge2['region'].replace('Могилевская','mogilevskaya')
df_merge2['region']=df_merge2['region'].replace('город Минск','minsk')
df_merge2.region=df_merge2.region.replace(0, 'no_region')
# df_merge3=pd.get_dummies(df_merge2, 'region')

In [ ]:
df_merge2.info()
df_merge_d=df_merge2.fillna(0)
df_merge_d.to_excel("df_merge_d.xlsx")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

In [ ]:

# обязательный код
X=df_merge_d.drop(columns=['lukashenko_share','Dmitriev','Kanopatskaya','Lukashenko','Tikhanovskaya','Cherechen']).values
y=df_merge_d.lukashenko_share.values
# сплитуйте X и y как хотите
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, 
                                                    shuffle=True)

In [ ]:
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1600, num = 8)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 90, num = 9)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
%%R

install.packages("skimr")
install.packages("glmnet")
install.packages("plotmo")

library(readxl)
library(MASS)
library(tidyverse)
library(broom)
library(skimr)
library(glmnet)
library(plotmo)
set.seed(144)

In [ ]:
%%R
data <- read_excel("df_merge_d.xlsx", sheet = "Sheet1")

p = ncol(data) -1
n <- nrow(data)
colnames(data) = c("total", "")
ind_train = sample(x = 1:n, size = ceiling(0.66 * n))
set_train = data[ind_train,]
ind_test = setdiff(x=1:n, ind_train)
set_test = data[ind_test,] 
head(data)

In [ ]:
%%R
summary(set_test)

In [ ]:
%%R
par(mfrow = c(3, 1))
hist(data$lukashenko_share) 
hist(set_train$lukashenko_share)
hist(set_test$lukashenko_share)

In [ ]:
%%R
skim(data)

In [ ]:
%%R
install.packages("corrplot")
library(corrplot)
correl = cor(data[-1])
corrplot(correl)

In [ ]:
model_lasso = glmnet(x = as.matrix(set_train[, - (p+1)]), y = set_train$lukashenko_share, alpha =1)

cv<-cv.glmnet(as.matrix(set_train[-(p+1)]), set_train$lukashenko_share, nfolds = 3) 
plot(cv)

lambda_lasso = cv.glmnet(x = as.matrix(set_train[,-(p+1)]),y = set_train$lukashenko_share, alpha =1)$lambda.min

# Plot log lambda
plot_glmnet(x = model_lasso, label = TRUE, xvar = "lambda")
title(main = "LASSO", line = 3)


We can first look at how our models perform on our training data.
Here we are comparing a random forest and a lasso model. 

In [ ]:
y_train = set_train$lukashenko_share
predict_train = matrix(data =0, nrow= nrow(set_train), ncol=2)

predict_train[, 1] = predict(object = model_naive, newdata = set_train[, -(p + 1)])
predict_train[,2] = predict.glmnet(object = model_lasso, 
                                  newx = as.matrix(set_train[,-(p+1)]),
                                  s = lambda_lasso)

colnames(predict_train) =c("Naive Model", "Lasso Model")

Predicting on our training data, we can see that the OLS model without regularization performs better than the Lasso model, although they are somewhat similar. 
Subsequently, we can look at how our model performs when dealing with new data. 
Therefore, we use the test set and again predict our naive and our lasso model.

In [ ]:
y_test = set_test$lukashenko_share
predict_test = matrix(data =0, nrow= nrow(set_test), ncol=2)

predict_test[, 1] = predict(object = model_naive, newdata = set_test[, -(p + 1)])
predict_test[,2] = predict.glmnet(object = model_lasso,  newx = as.matrix(set_test[,-(p+1)]),
                                  s = lambda_lasso)

colnames(predict_test) =c("Naive Model", "Lasso Model")

MSE_test = rep(x=0, length.out =2) 
for (i in 1:2){
  MSE_test[i] = mean((y_test-predict_test[,i])^2)
}
names(MSE_test) = c("Naive Model", "Lasso Model")
MSE_test

# OLS Regression with selected features

Here we can see a very different performance. 
The Lasso model performs substantially better then the OLS model.
Therefore, using this method has led to better predictive performance. 
Lastly, we can also use the lasso specification for a OLS model, trained on test data to ensure that we have no incorrect standard errors.
This is a pivotal concern within in economics, because we need correct standard errors to interpret our coefficients and ensure causality. 

As a first step we can look at the coefficients in the lasso model:

In [ ]:
coef_lasso <- model_lasso$beta[, which(model_lasso$lambda == lambda_lasso)]
which(coef_lasso!=0)

The advantage of the lasso method is that we induce sparsity.
Here we can see that various features turn to 0.
One could also argue that due to correlation between some features, explanatory or variance was ascribed to either the one or the other variable. 
We can see that the variables that were more correlated (see correlation plot above) frequently scored 0 here. 
Now we can specify a model with the relevant coefficients, and run the model on the test data. 

In [ ]:
model = lm(lukashenko_share ~  earlyvoting  + residence + electday + dropped + against_all + commission + spoiled +  long + army + educ + science + profunion + econ, data = set_test)
summary(model)

# Feature Importance for Lasso Regression 
 
 For comparability, we also include 

# Alternative Methods: Unsupervised Learning via Principal Component Analysis 

In addition to supervised learning, we can also look at feature importance from the perspective of unsupervised learning.
One popular dimensionality reduction technique is principal component analysis or in short PCA. 


In [ ]:
library(ggfortify)
library(devtools)
install_github("vqv/ggbiplot")
library(ggbiplot)

summary(PCA)
screeplot(PCA, npcs =8, type = "lines")
install.packages("ggfortify")
install.packages("")
autoplot(PCA, colour = "lukashenko_share", loadings =T,loading.label =T,loadings.label.size = 20)

In [ ]:
PCA = prcomp(data[,19:31], scale. = TRUE,center = TRUE)
summary(PCA)
ggbiplot(PCA,ellipse=TRUE, labels = rownames(data), groups= data$regions)

# Interpretation 

Here one can see (#TODO)

# Alternative Methods: Unsupervised Learning via Latent Dirichlet Allocation 



In [ ]:
%%R
install.packages("lda")
install.packages("topicmodels")
install.packages("tm")

library(tidyverse)
library(lda)
library(topicmodels)
library(tidytext)
library(tm)

#data = unite(lits3_gis,col = "text", country:b2c, sep =" ")
#data <- read_csv("~/Downloads/BelarusElections/belarus-qgis.csv")
data <- read_excel("df_merge_n.xlsx", sheet = "Sheet1")
data = unite(data, col = "text", commission_code:economics, sep =" ")

In [ ]:
data = data %>%
  select("text")

cleaner <- function(text){
  text <- tolower(text)
  text <- gsub("rt", "", text)
  text <- gsub("@\\w+", "", text)
  text <- gsub("[[:punct:]]", "", text)
  text <- gsub("http\\w+", "", text)
  text <- gsub("amp", " ", text)
  text <- gsub("[ |\t]{2,}", "", text)
  text <- gsub("^ ", "", text)
  text <- gsub(" $", "", text)
  text <- gsub(" +", " ", text)
  text <- gsub("=", " ", text)
  text <- gsub('<.*>', '', enc2native(text))
  text <- unique(text)
  return(text)
}
#####################################################################################

polish <- function(text){
  text <- VCorpus(VectorSource(text))
  text <- tm_map(text, removeWords, stopwords("english"))
  text <- tm_map(text, removeNumbers)
  text <- tm_map(text, stemDocument)
}

In [ ]:
text = data$text 
text <- cleaner(text)
corpus <- polish(text)

In [ ]:
doc.lengths <- rowSums(as.matrix(DocumentTermMatrix(corpus)))
dtm <- DocumentTermMatrix(corpus[doc.lengths > 0])

#####################################################################################

#LDA <- LDA(dtm, k =3,method = "VEM", cotrol = list(seed=1234))
LDA <- LDA(x=dtm, k=2, method="Gibbs",control=list(alpha=1, delta=0.1, seed=10005))

topics <- tidy(LDA, matrix ="beta")

ap_top_terms <- topics %>%
  group_by(topic) %>%
  top_n(10, beta) %>%
  ungroup() %>%
  arrange(topic, -beta)

#####################################################################################

ap_top_terms %>%
  mutate(term = reorder_within(term, beta, topic)) %>%
  ggplot(aes(term, beta, fill = factor(topic))) +
  geom_col(show.legend = FALSE) +
  facet_wrap(~ topic, scales = "free") +
  coord_flip() +
  scale_x_reordered()

# Geospatial Analysis 

In [ ]:
%%R
library(ggplot2)
library(sf)
library(maps)

votes <- read.csv("~/Downloads/BelarusElections/belarus1994-qgisNew.csv",header=T)
geo = st_read("~/Downloads/BelarusElections 4/gis files for BLR and poland/BLR_Adm/BLR_adm2.shp")
dist = st_read("~/Downloads/BelarusElections 4/gis files for BLR and poland/BLR_Adm/BLR_adm1.shp")
road = st_read("~/Downloads/BelarusElections 4/gis files for BLR and poland/BLR_rds/BLR_roads.shp")
bela <- read.csv("~/Downloads/BelarusElections 3/belarus-qgis.csv", header=T)
data <- read_excel("df_merge_n.xlsx", sheet = "Sheet1")

plot = ggplot()+ geom_sf(data = geo, mapping = aes(geometry = geometry)) + 
  geom_sf(data= dist, aes(geometry = geometry)) + 
  geom_sf(data= road, aes(geometry = geometry)) 

map = plot + geom_point(data = bela, mapping = aes(fill = "region")) + coord_sf()
map
